In [1]:
import pandas as pd

In [2]:
pd.__version__

'0.19.0'

In [3]:
df_business = pd.read_json('../../dataset/academic/yelp_academic_dataset_business.json', lines=True)

In [4]:
len(df_business)

85901

In [5]:
df_business_restaurants = df_business[df_business['categories'].apply(lambda x: 'Restaurants' in x)].reset_index(drop=True)
#df_business_restaurants

In [6]:
df_business_restaurants.drop(['latitude','longitude','neighborhoods','type','full_address'], axis=1, inplace=True)

In [7]:
df_business_restaurants_open = df_business_restaurants[df_business_restaurants['open']==True].reset_index(drop=True)
df_business_restaurants_open
len(df_business_restaurants_open)

20125

In [8]:
df_business_restaurants_open.columns

Index(['attributes', 'business_id', 'categories', 'city', 'hours', 'name',
       'open', 'review_count', 'stars', 'state'],
      dtype='object')

In [9]:
uniqueattributes = []
allattributes = df_business_restaurants_open['attributes']
for attributes in allattributes.iteritems():
    uniqueattributes.extend(attributes[1])
uniqueattributes_set = set(uniqueattributes)
uniqueattributes_set

{'Accepts Credit Cards',
 'Accepts Insurance',
 'Ages Allowed',
 'Alcohol',
 'Ambience',
 'Attire',
 'BYOB',
 'BYOB/Corkage',
 'By Appointment Only',
 'Caters',
 'Coat Check',
 'Corkage',
 'Delivery',
 'Dietary Restrictions',
 'Dogs Allowed',
 'Drive-Thru',
 'Good For',
 'Good For Dancing',
 'Good For Groups',
 'Good for Kids',
 'Happy Hour',
 'Has TV',
 'Music',
 'Noise Level',
 'Open 24 Hours',
 'Order at Counter',
 'Outdoor Seating',
 'Parking',
 'Price Range',
 'Smoking',
 'Take-out',
 'Takes Reservations',
 'Waiter Service',
 'Wheelchair Accessible',
 'Wi-Fi'}

In [10]:
df_business_restaurants_open['stars_round'] = df_business_restaurants_open['stars'].apply(round)

In [11]:
df_business_restaurants_open.head()

,attributes,business_id,categories,city,hours,name,open,review_count,stars,state,stars_round
0,"{'Alcohol': 'none', 'Good for Kids': True, 'At...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"{'Thursday': {'open': '11:00', 'close': '21:00...",Mr Hoagie,True,7,3.5,PA,4
1,"{'Alcohol': 'full_bar', 'Smoking': 'no', 'Atti...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"{'Wednesday': {'open': '10:00', 'close': '19:0...",Emil's Lounge,True,26,4.5,PA,4
2,"{'Alcohol': 'full_bar', 'Smoking': 'outdoor', ...",KayYbHCt-RkbGcPdGOThNg,"[Bars, American (Traditional), Nightlife, Rest...",Carnegie,"{'Wednesday': {'open': '11:00', 'close': '02:0...",Alexion's Bar & Grill,True,23,4.0,PA,4
3,"{'Wheelchair Accessible': True, 'Alcohol': 'no...",wJr6kSA5dchdgOdwH6dZ2w,"[Burgers, Breakfast & Brunch, American (Tradit...",Carnegie,"{'Wednesday': {'open': '08:00', 'close': '02:0...",Kings Family Restaurant,True,10,3.5,PA,4
4,"{'Alcohol': 'full_bar', 'Good for Kids': True,...",fNGIbpazjTRdXgwRY_NIXA,"[Bars, American (Traditional), Nightlife, Loun...",Carnegie,"{'Wednesday': {'open': '11:00', 'close': '23:0...",Rocky's Lounge,True,10,4.0,PA,4


# Starting analysis for Accepts Credit Cards

In [33]:
df_restaurants_accepts_creditcards = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Accepts Credit Cards']]

df_restaurants_accepts_creditcards['Accepts Credit Cards'].unique()

#accept_credit_values = [True,False]

array([True, False, nan], dtype=object)

In [13]:
#dictionary for storing final star against creditcard frequency
stars_creditcard_dict = {}
# group business by stars
group_by_stars = df_restaurants_accepts_creditcards.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store creditcard value and frequency
    creditcard_dict = {}
    # This is to find the nmber of 'Unknown' credit card category
    total_bus_credit_info = 0
    for accept_credit_value in accept_credit_values:
        bus_count = len(dataforeachstar[dataforeachstar['Accepts Credit Cards'].apply(lambda x: accept_credit_value == x)].
                        reset_index(drop=True))
        creditcard_dict[accept_credit_value] = (bus_count + 1)/(star_bus_count + 3)
        total_bus_credit_info += bus_count
    creditcard_dict['Unknown'] = (star_bus_count - total_bus_credit_info + 1)/(star_bus_count + 3)
    stars_creditcard_dict[group] = creditcard_dict
#Result
stars_creditcard_df = pd.DataFrame.from_dict(stars_creditcard_dict)

In [14]:
stars_creditcard_df

,1,2,3,4,5
False,0.011111,0.006915,0.010749,0.023047,0.061275
True,0.677778,0.825316,0.814189,0.765084,0.571078
Unknown,0.311111,0.167769,0.175061,0.211868,0.367647


# Starting analysis for Alcohol

In [15]:
df_restaurants_alcohol = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Alcohol']]
alcohol_values = ['none', 'full_bar','beer_and_wine']

In [16]:
#dictionary for storing final star against alcohol frequency
stars_alcohol_dict = {}
# group business by stars
group_by_stars = df_restaurants_alcohol.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store alcohol value and frequency
    alcohol_dict = {}
    # This is to find the nmber of 'Unknown' alcohol category
    total_bus_alcohol_info = 0
    for alcohol_value in alcohol_values:
        bus_count = len(dataforeachstar[dataforeachstar['Alcohol'].apply(lambda x: alcohol_value == x)].
                        reset_index(drop=True))
        alcohol_dict[alcohol_value] = (bus_count + 1)/(star_bus_count + len(alcohol_values)+1)
        total_bus_alcohol_info += bus_count
    alcohol_dict['Unknown'] = (star_bus_count - total_bus_alcohol_info + 1)/(star_bus_count + len(alcohol_values)+1)
    stars_alcohol_dict[group] = alcohol_dict
#Result
stars_alcohol_df = pd.DataFrame.from_dict(stars_alcohol_dict)

In [17]:
stars_alcohol_df

,1,2,3,4,5
Unknown,0.637363,0.272919,0.165797,0.133221,0.354523
beer_and_wine,0.021978,0.055005,0.115137,0.152821,0.102689
full_bar,0.021978,0.153592,0.311636,0.332746,0.112469
none,0.318681,0.518485,0.407430,0.381211,0.430318


# Starting analysis for  caters

In [18]:
df_restaurants_caters = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Caters']]
caters_values = [False, True]

In [19]:
#dictionary for storing final star against caters frequency
stars_caters_dict = {}
# group business by stars
group_by_stars = df_restaurants_caters.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store caters value and frequency
    caters_dict = {}
    # This is to find the nmber of 'Unknown' caters category
    total_bus_caters_info = 0
    for caters_value in caters_values:
        bus_count = len(dataforeachstar[dataforeachstar['Caters'].apply(lambda x: caters_value == x)].
                        reset_index(drop=True))
        caters_dict[caters_value] = (bus_count + 1)/(star_bus_count + len(caters_values)+1)
        total_bus_caters_info += bus_count
    caters_dict['Unknown'] = (star_bus_count - total_bus_caters_info + 1)/(star_bus_count + len(caters_values)+1)
    stars_caters_dict[group] = caters_dict
#Result
stars_caters_df = pd.DataFrame.from_dict(stars_caters_dict)

In [20]:
stars_caters_df

,1,2,3,4,5
False,0.255556,0.419723,0.38145,0.336371,0.178922
True,0.100000,0.163860,0.30129,0.393262,0.269608
Unknown,0.644444,0.416416,0.31726,0.270368,0.551471


# Starting analysis for  Noise level

In [21]:
df_restaurants_noise_level = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Noise Level']]
noise_level_values = ['average', 'loud','quiet', 'very_loud']

In [22]:
#dictionary for storing final star against noise_level frequency
stars_noise_level_dict = {}
# group business by stars
group_by_stars = df_restaurants_noise_level.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store noise_level value and frequency
    noise_level_dict = {}
    # This is to find the nmber of 'Unknown' noise_level category
    total_bus_noise_level_info = 0
    for noise_level_value in noise_level_values:
        bus_count = len(dataforeachstar[dataforeachstar['Noise Level'].apply(lambda x: noise_level_value == x)].
                        reset_index(drop=True))
        noise_level_dict[noise_level_value] = (bus_count + 1)/(star_bus_count + len(noise_level_values)+1)
        total_bus_noise_level_info += bus_count
    noise_level_dict['Unknown'] = (star_bus_count - total_bus_noise_level_info + 1)/(star_bus_count + len(noise_level_values)+1)
    stars_noise_level_dict[group] = noise_level_dict
#Result
stars_noise_level_df = pd.DataFrame.from_dict(stars_noise_level_dict)

In [23]:
stars_noise_level_df

,1,2,3,4,5
Unknown,0.673913,0.331130,0.195826,0.154417,0.451220
average,0.163043,0.395132,0.546041,0.606339,0.295122
loud,0.065217,0.087139,0.085635,0.051141,0.024390
quiet,0.076087,0.137620,0.142112,0.174169,0.224390
very_loud,0.021739,0.048978,0.030387,0.013934,0.004878


# Starting analysis for Price Range

In [24]:
df_restaurants_pricerange = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Price Range']]
pricerange_values = [1., 2., 3., 4.]

In [25]:
#dictionary for storing final star against pricerange frequency
stars_pricerange_dict = {}
# group business by stars
group_by_stars = df_restaurants_pricerange.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store pricerange value and frequency
    pricerange_dict = {}
    # This is to find the nmber of 'Unknown' pricerange category
    total_bus_pricerange_info = 0
    for pricerange_value in pricerange_values:
        bus_count = len(dataforeachstar[dataforeachstar['Price Range'].apply(lambda x: pricerange_value == x)].
                        reset_index(drop=True))
        pricerange_dict[pricerange_value] = (bus_count + 1)/(star_bus_count + len(pricerange_values)+1)
        total_bus_pricerange_info += bus_count
    pricerange_dict['Unknown'] = (star_bus_count - total_bus_pricerange_info + 1)/(star_bus_count + len(pricerange_values)+1)
    stars_pricerange_dict[group] = pricerange_dict
#Result
stars_pricerange_df = pd.DataFrame.from_dict(stars_pricerange_dict)

In [26]:
stars_pricerange_df

,1,2,3,4,5
1.0,0.391304,0.560397,0.446900,0.405298,0.443902
2.0,0.228261,0.317308,0.465009,0.475808,0.295122
3.0,0.010870,0.019231,0.033763,0.059868,0.078049
4.0,0.010870,0.006010,0.003990,0.013091,0.029268
Unknown,0.358696,0.097055,0.050338,0.045935,0.153659


# Starting analysis for Take-out

In [27]:
df_restaurants_takeout = pd.concat([df_business_restaurants_open[['business_id','stars_round']], 
           df_business_restaurants_open['attributes'].apply(pd.Series)], axis=1)[['business_id','stars_round','Take-out']]
takeout_values = [True,False]

In [28]:
#dictionary for storing final star against takeout frequency
stars_takeout_dict = {}
# group business by stars
group_by_stars = df_restaurants_takeout.groupby('stars_round')
# loop to find info in ech star group
for group in group_by_stars.groups:
    # no. of businesses in this star group
    star_bus_count = group_by_stars.get_group(group)['business_id'].count()
    dataforeachstar = group_by_stars.get_group(group)
    # dictionary to store takeout value and frequency
    takeout_dict = {}
    # This is to find the nmber of 'Unknown' takeout category
    total_bus_takeout_info = 0
    for takeout_value in takeout_values:
        bus_count = len(dataforeachstar[dataforeachstar['Take-out'].apply(lambda x: takeout_value == x)].
                        reset_index(drop=True))
        takeout_dict[takeout_value] = (bus_count + 1)/(star_bus_count + len(takeout_values)+1)
        total_bus_takeout_info += bus_count
    takeout_dict['Unknown'] = (star_bus_count - total_bus_takeout_info + 1)/(star_bus_count + len(takeout_values)+1)
    stars_takeout_dict[group] = takeout_dict
#Result
stars_takeout_df = pd.DataFrame.from_dict(stars_takeout_dict)

In [29]:
stars_takeout_df

,1,2,3,4,5
False,0.033333,0.050511,0.063882,0.095176,0.120098
True,0.666667,0.862898,0.883600,0.856968,0.759804
Unknown,0.300000,0.086590,0.052518,0.047856,0.120098


In [30]:
print ("\n ******* Credit cards accepted *****")
print (stars_creditcard_df)
print ("\n ****** Alcohol ***** ")
print (stars_alcohol_df)
print ("\n ****** Caters ********")
print (stars_caters_df)
print ("\n ********* Noise level*******")
print (stars_noise_level_df)
print ("\n *********** Price range ********")
print (stars_pricerange_df)
print ("\n ********* Take out ***********")
print (stars_takeout_df)


 ******* Credit cards accepted *****
                1         2         3         4         5
False    0.011111  0.006915  0.010749  0.023047  0.061275
True     0.677778  0.825316  0.814189  0.765084  0.571078
Unknown  0.311111  0.167769  0.175061  0.211868  0.367647

 ****** Alcohol ***** 
                      1         2         3         4         5
Unknown        0.637363  0.272919  0.165797  0.133221  0.354523
beer_and_wine  0.021978  0.055005  0.115137  0.152821  0.102689
full_bar       0.021978  0.153592  0.311636  0.332746  0.112469
none           0.318681  0.518485  0.407430  0.381211  0.430318

 ****** Caters ********
                1         2        3         4         5
False    0.255556  0.419723  0.38145  0.336371  0.178922
True     0.100000  0.163860  0.30129  0.393262  0.269608
Unknown  0.644444  0.416416  0.31726  0.270368  0.551471

 ********* Noise level*******
                  1         2         3         4         5
Unknown    0.673913  0.331130  0.195826  0